# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 24 2022 3:15PM,240434,10,8188297,Bio-PRF,Released
1,Jun 24 2022 3:09PM,240433,12,HH-VegasOrder,Hush Hush,Executing
2,Jun 24 2022 3:00PM,240424,10,0085799193,ISDIN Corporation,Released
3,Jun 24 2022 3:00PM,240424,10,0085799188,ISDIN Corporation,Released
4,Jun 24 2022 3:00PM,240424,10,0085799192,ISDIN Corporation,Released
5,Jun 24 2022 3:00PM,240424,10,0085799189,ISDIN Corporation,Released
6,Jun 24 2022 3:00PM,240424,10,0085799196,ISDIN Corporation,Released
7,Jun 24 2022 3:00PM,240424,10,0085799194,ISDIN Corporation,Released
8,Jun 24 2022 3:00PM,240424,10,0085799191,ISDIN Corporation,Released
9,Jun 24 2022 3:00PM,240424,10,0085799186,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,240428,Released,1
14,240429,Released,5
15,240430,Released,1
16,240433,Executing,1
17,240434,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240428,NaN,NaN,1.0
240429,NaN,NaN,5.0
240430,NaN,NaN,1.0
240433,NaN,1.0,NaN
240434,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240348,0.0,0.0,1.0
240352,0.0,0.0,1.0
240355,0.0,0.0,1.0
240385,0.0,1.0,0.0
240386,0.0,0.0,36.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240348,0,0,1
240352,0,0,1
240355,0,0,1
240385,0,1,0
240386,0,0,36


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240348,0,0,1
1,240352,0,0,1
2,240355,0,0,1
3,240385,0,1,0
4,240386,0,0,36


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240348,,,1
1,240352,,,1
2,240355,,,1
3,240385,,1,
4,240386,,,36


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 24 2022 3:15PM,240434,10,Bio-PRF
1,Jun 24 2022 3:09PM,240433,12,Hush Hush
2,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation
56,Jun 24 2022 2:59PM,240423,20,Alumier Labs Inc.
76,Jun 24 2022 2:57PM,240430,19,ACG North America LLC
77,Jun 24 2022 2:56PM,240427,10,ISDIN Corporation
99,Jun 24 2022 2:54PM,240429,10,ISDIN Corporation
104,Jun 24 2022 2:53PM,240428,19,ACG North America LLC
105,Jun 24 2022 2:51PM,240422,10,ISDIN Corporation
141,Jun 24 2022 11:40AM,240404,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 24 2022 3:15PM,240434,10,Bio-PRF,,,1
1,Jun 24 2022 3:09PM,240433,12,Hush Hush,,1,
2,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,,,54
3,Jun 24 2022 2:59PM,240423,20,Alumier Labs Inc.,,,20
4,Jun 24 2022 2:57PM,240430,19,ACG North America LLC,,,1
5,Jun 24 2022 2:56PM,240427,10,ISDIN Corporation,,11,11
6,Jun 24 2022 2:54PM,240429,10,ISDIN Corporation,,,5
7,Jun 24 2022 2:53PM,240428,19,ACG North America LLC,,,1
8,Jun 24 2022 2:51PM,240422,10,ISDIN Corporation,,,36
9,Jun 24 2022 11:40AM,240404,10,ISDIN Corporation,,,35


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 24 2022 3:15PM,240434,10,Bio-PRF,1,,
1,Jun 24 2022 3:09PM,240433,12,Hush Hush,,1,
2,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,54,,
3,Jun 24 2022 2:59PM,240423,20,Alumier Labs Inc.,20,,
4,Jun 24 2022 2:57PM,240430,19,ACG North America LLC,1,,
5,Jun 24 2022 2:56PM,240427,10,ISDIN Corporation,11,11,
6,Jun 24 2022 2:54PM,240429,10,ISDIN Corporation,5,,
7,Jun 24 2022 2:53PM,240428,19,ACG North America LLC,1,,
8,Jun 24 2022 2:51PM,240422,10,ISDIN Corporation,36,,
9,Jun 24 2022 11:40AM,240404,10,ISDIN Corporation,35,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 24 2022 3:15PM,240434,10,Bio-PRF,1,,
1,Jun 24 2022 3:09PM,240433,12,Hush Hush,,1,
2,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,54,,
3,Jun 24 2022 2:59PM,240423,20,Alumier Labs Inc.,20,,
4,Jun 24 2022 2:57PM,240430,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 24 2022 3:15PM,240434,10,Bio-PRF,1.0,NaN,NaN
1,Jun 24 2022 3:09PM,240433,12,Hush Hush,NaN,1.0,NaN
2,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,54.0,NaN,NaN
3,Jun 24 2022 2:59PM,240423,20,Alumier Labs Inc.,20.0,NaN,NaN
4,Jun 24 2022 2:57PM,240430,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 24 2022 3:15PM,240434,10,Bio-PRF,1.0,0.0,0.0
1,Jun 24 2022 3:09PM,240433,12,Hush Hush,0.0,1.0,0.0
2,Jun 24 2022 3:00PM,240424,10,ISDIN Corporation,54.0,0.0,0.0
3,Jun 24 2022 2:59PM,240423,20,Alumier Labs Inc.,20.0,0.0,0.0
4,Jun 24 2022 2:57PM,240430,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1682926,178.0,11.0,0.0
12,480818,0.0,2.0,0.0
16,721055,3.0,0.0,0.0
19,480858,2.0,0.0,0.0
20,480828,20.0,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1682926,178.0,11.0,0.0
1,12,480818,0.0,2.0,0.0
2,16,721055,3.0,0.0,0.0
3,19,480858,2.0,0.0,0.0
4,20,480828,20.0,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,178.0,11.0,0.0
1,12,0.0,2.0,0.0
2,16,3.0,0.0,0.0
3,19,2.0,0.0,0.0
4,20,20.0,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,178.0
1,12,Released,0.0
2,16,Released,3.0
3,19,Released,2.0
4,20,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Completed,0.0,0.0,0.0,0.0,1.0
Executing,11.0,2.0,0.0,0.0,1.0
Released,178.0,0.0,3.0,2.0,20.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Completed,0.0,0.0,0.0,0.0,1.0
1,Executing,11.0,2.0,0.0,0.0,1.0
2,Released,178.0,0.0,3.0,2.0,20.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Completed,0.0,0.0,0.0,0.0,1.0
1,Executing,11.0,2.0,0.0,0.0,1.0
2,Released,178.0,0.0,3.0,2.0,20.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()